<a href="https://colab.research.google.com/github/fatihkykc/lda4wpchat/blob/master/LDAForWhatsappChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install emoji
!pip install git+https://github.com/emres/turkish-deasciifier.git


In [0]:

import pickle
from google.colab import files
from preprocess import preProcessing
from gensim.models import TfidfModel, LdaModel, CoherenceModel
from gensim.corpora import Dictionary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
uploaded = files.upload()

In [0]:
preprocessing = preProcessing()
df = preprocessing.process('/content',False, False, True)
dictionary = Dictionary(df['msg'])

In [0]:
df['datehour'] = df['date'].dt.hour
df['dates'] = df['date'].dt.date
df['datemins'] = df['date'].dt.minute
df['decile'] = df['datemins'].apply(lambda x: int(x//10))
new = df.groupby(['conv_name','datehour','decile', 'dates', 'group'])['msg'].sum().reset_index()
new.tail(50)

In [0]:
df['date'].dt.date

In [0]:
#build b2w corpus
bow_corpus = [dictionary.doc2bow(document) for document in df['msg']]

#calculate tfidf
tf_idf = TfidfModel(bow_corpus)
corpus_tf_idf = tf_idf[bow_corpus]  

In [0]:
for idx, doc in enumerate(df['msg']):
  for token in doc:
    if "http" in token:
      print(doc, idx)

In [0]:
lda_models_with_coherence_score = {}
for index in range(1, 10 + 1):
    lda_model = LdaModel(corpus=corpus_tf_idf,
                         num_topics=index,
                         id2word=dictionary)

    coherence_model_lda = CoherenceModel(model=lda_model,
                                         texts=df['msg'],
                                         corpus=corpus_tf_idf,
                                         coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    lda_models_with_coherence_score[coherence_score] = lda_model

In [0]:
lda_model = LdaModel(corpus=corpus_tf_idf,
                         num_topics=150,
                         id2word=dictionary,
                         random_state=100,
                         update_every=1,
                         chunksize=100,
                         passes=10,
                         alpha=0.25,
                         per_word_topics=True
                     )

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [0]:
#lda_model = lda_models_with_coherence_score[max(lda_models_with_coherence_score)]

with open('./lda_v2.model', "wb") as f:
 pickle.dump(lda_model, f)
 f.close()

lda_model.show_topics(num_topics=150,num_words=15)

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_tf_idf))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['msg'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -120.2367816937994

Coherence Score:  0.37224970630462595
